# 对抗攻击

## 导入需要的包

In [1]:
import torch
from torch.utils import data
from torch import nn
from torchvision import datasets

import utils
import attack

import pandas as pd
import numpy as np
import time
import os
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm_notebook as tqdm
from torch.utils.tensorboard import SummaryWriter


## 性能对比

### 定义需要的函数

In [ ]:
def plot_imgs(att_mode: str, examples, epsilons, num_examples: int = 5):
    plt.figure(att_mode, figsize=(30, 30))
    cnt = 0
    for eps in epsilons:
        ex = examples[att_mode][eps]
        num_ex_imgs = len(ex['imgs'])
        for idx in range(num_examples):
            cnt += 1
            if idx >= num_ex_imgs:
                continue
            plt.subplot(len(epsilons), num_examples, cnt)
            plt.xticks([], [])
            plt.yticks([], [])
            if idx == 0:
                plt.ylabel(f'Eps: {eps:.2f}', fontsize=20)
            suc_label, suc_prob = ex['suc_labels'][idx], ex['suc_probs'][idx]
            att_pred, att_prob = ex['att_preds'][idx], ex['att_probs'][idx]

            suc_tag = utils.class_names[dataset_name][suc_label]
            att_tag = utils.class_names[dataset_name][att_pred]

            img = ex['imgs'][idx]
            img_np = img.permute(1, 2, 0).numpy()

            plt.title(f'{suc_tag}({suc_prob*100:4.2f}%) -> {att_tag}({att_prob*100:4.2f}%)', fontsize=20)
            plt.imshow(img_np)    

    plt.tight_layout(w_pad=0)
    # plt.show()

### 设置基本参数

In [ ]:
# 设置随机种子，使结果可复现
seed = 0
utils.set_random_seed(seed)

# 设置在GPU上运行
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 设置模型名称和路径
model_name = 'data-large-alexnet'
# model_path = r'./results/CIFAR10/pre-resnet-bs128-lr0.01-seed0/models/pre-resnet-bs128-lr0.01-seed0'
# model_path = r'./results/CIFAR10/pre-mobilenet-bs64-lr0.01-seed0/models/pre-mobilenet-bs64-lr0.01-seed0'
model_path = r'./results/CIFAR10/alexnet-bs128-lr0.01-seed0/models/alexnet-bs128-lr0.01-seed0'

# 设置数据集相关参数
dataset_name = 'CIFAR10'# 数据集类型
data_channel = 3        # 图像通道数
batch_size = 100        # 图片批量大小
max_iter = 10           # 最大迭代的批量数量，减少程序运行时间

# 攻击强度范围和步长
# epsilons = np.arange(0., 0.15, 0.015)
epsilons = [0.15, 0.2, 0.25, 0.3, 0.4, 0.5]


# 设置结果参数
num_examples = 5        # 每种参数下的示例图片数量
metrics = {}            # 相关数据
examples = {}           # 示例图片

# 设置损失函数，用于计算损失值
loss_fn = nn.CrossEntropyLoss()

# 设置保存路径
att_results_path = f'./results_attack/{dataset_name}/{model_name}'
att_metrics_path = f'{att_results_path}/metrics/'
att_examples_path = f'{att_results_path}/examples/'
att_tb_path = f'{att_results_path}/tb_out/'
paths = [att_metrics_path, att_tb_path, att_examples_path]
for path in paths:
    # 若路径已存在则不创建新文件夹
    if not os.path.exists(path):
        os.makedirs(path)

# 检查相应路径下是否已存在数据
utils.check_path(att_results_path)

# 设置tensorboard可视化文件的保存路径
tb_writer = SummaryWriter(att_tb_path)

### 创建待评估模型与数据集

In [ ]:
# 读取路径下的模型
model = utils.creat_model(model_name, model_path, in_ch=data_channel, pretrained=True)
# 创建测试数据集
test_data = utils.data_iter(dataset_name, 'test', batch_size, 0)[1]

### 设置攻击类型

In [ ]:
# None/FGSM/IFGSM
att_modes = []
# att_modes.append('None')
att_modes.append('FGSM')
att_modes.append('PGD')
att_modes.append('IFGSM')
att_modes.append('NIFGSM')
# att_modes.append('DeepFool')
att_modes

### 无对抗攻击

In [ ]:
if att_modes == 'None':
    # 测试无对抗攻击时的模型精度
    loss, top1_acc = utils.test_accuracy(model, test_data)
    print(f'loss: {loss:.2e}\tTop 1 acc: {top1_acc * 100:4.2f}%')

    # 清空显存缓存，防止显存爆炸
    torch.cuda.empty_cache()

### FGSM

In [ ]:
if 'FGSM' in att_modes:
    # 临时变量，临时保存数据与示例图片
    metrics_temp = {}
    examples_temp = {}

    # 新建攻击模型，并设置名称
    att_method = attack.FGSM(model)
    att_name = att_method.__class__.__name__

    # tensorboard所需要的步数
    step = 0
    
    # 循环遍历攻击强度，得到每种强度下的相关数据
    with tqdm(epsilons, total = len(epsilons), leave = True) as t_eps:
        for eps in t_eps:
            # 定义相关参数累加器
            att_accu = utils.Accumulator(5)
            
            # 定义当前数据集批量迭代次数
            num_iters = 0
            with tqdm(test_data, total = max_iter, leave = False) as t_data:
                for imgs_in, labels in t_data:
                    # 当迭代次数超过设定值时，停止迭代，退出循环
                    num_iters += 1
                    if num_iters > max_iter: break
                    
                    # 提前预测一遍数据集，挑选出预测成功的例子用于攻击
                    raw_preds, raw_probs, Y_hat = utils.predict(model, imgs_in, dataset_name)

                    # 计算损失值
                    loss = loss_fn(Y_hat, labels)

                    # 获取预测成功的标签
                    suc_indices = raw_preds == labels
                    suc_imgs, suc_labels, suc_probs = imgs_in[suc_indices], labels[suc_indices], raw_probs[suc_indices]


                    #----------------------------------------#
                    # 开始攻击
                    #----------------------------------------#
                    
                    attacks = torch.zeros_like(suc_imgs)

                    # 计算时间
                    time_start = time.time()
                    # 攻击图片
                    attacks = att_method(suc_imgs, suc_labels, eps)
                    time_end = time.time()

                    # 得到扰动图片
                    deltas = attacks - suc_imgs
                    # 计算扰动的大小，L2范数
                    att_norm = torch.mean(
                    torch.linalg.norm(deltas.reshape(len(deltas), -1), dim=1) 
                        / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim=1)
                    )

                    # 计算攻击后的数据
                    att_preds, att_probs, att_Y_hat = utils.predict(model, attacks, dataset_name)
                    att_loss = loss_fn(att_Y_hat, suc_labels)
                    att_suc_rate = (att_preds != suc_labels).sum() / len(att_preds)
                    att_acc = 1 - att_suc_rate
                    att_time = (time_end - time_start)/len(suc_imgs)

                    # 累加相关数据，用于求平均值
                    att_accu.add(att_acc, att_suc_rate, att_loss, att_norm, att_time)                    
            
            if eps == 0:    # 保存eps为0的特例
                att_suc_indices = att_preds == suc_labels
            else:           # 保存eps不为0时，攻击成功的标号
                att_suc_indices = att_preds != suc_labels

            # 示例数据，保存为字典
            examples_temp[eps] = {
                'suc_labels': suc_labels[att_suc_indices][0:num_examples].detach().cpu(),
                'suc_probs':suc_probs[att_suc_indices][0:num_examples].detach().cpu(),
                'att_preds':att_preds[att_suc_indices][0:num_examples].detach().cpu(),
                'att_probs':att_probs[att_suc_indices][0:num_examples].detach().cpu(),
                'imgs':attacks[att_suc_indices][0:num_examples].detach().cpu(),
                'deltas':deltas[att_suc_indices][0:num_examples].detach().cpu()
            }
            # 按攻击方式保存示例
            examples[att_name] = examples_temp

            # tensorboard可视化
            tb_writer.add_images(att_name, attacks, step)
            step += 1

            # 保存相关数据
            num_batch = min(max_iter, len(test_data))
            metrics_temp[eps] = {
                '模型精度'  : att_accu[0]/num_batch,
                '攻击成功率': att_accu[1]/num_batch,
                '损失大小'  : att_accu[2]/num_batch,
                '扰动大小'  : att_accu[3]/num_batch,
                '攻击时间'  : att_accu[4]/num_batch
            }
            metrics[att_name] = metrics_temp
            # print(f'{att_name}-{eps}:\n{metrics_temp}')

            # 清理显存缓存，防止显存爆炸
            torch.cuda.empty_cache()

    # 显示图片
    plot_imgs(att_name, examples, epsilons)
    

### BIM

In [ ]:
if 'IFGSM' in att_modes:
    metrics_temp = {}
    examples_temp = {}

    att_method = attack.BIM(model)
    att_name = att_method.__class__.__name__

    step = 0
    with tqdm(epsilons, total = len(epsilons), leave = True) as t_eps:
        for eps in t_eps:
            att_accu = utils.Accumulator(5)
            
            num_iters = 0
            with tqdm(test_data, total = max_iter, leave = False) as t_data:
                for imgs_in, labels in t_data:
                    num_iters += 1
                    if num_iters > max_iter: break

                    #----------------------------------------#
                    # 准备数据
                    #----------------------------------------#

                    raw_preds, raw_probs, Y_hat = utils.predict(model, imgs_in, dataset_name)

                    loss = loss_fn(Y_hat, labels)

                    suc_indices = raw_preds == labels
                    suc_imgs, suc_labels, suc_probs = imgs_in[suc_indices], labels[suc_indices], raw_probs[suc_indices]
                    
                    
                    #----------------------------------------#
                    # 开始攻击
                    #----------------------------------------#

                    attacks = torch.zeros_like(suc_imgs)

                    time_start = time.time()
                    attacks = att_method(suc_imgs, suc_labels, eps)
                    # BUG
                    # attacks = suc_imgs
                    time_end = time.time()

                    deltas = attacks - suc_imgs
                    att_norm = torch.mean(
                    torch.linalg.norm(deltas.reshape(len(deltas), -1), dim=1) 
                        / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim=1)
                    )


                    att_preds, att_probs, att_Y_hat = utils.predict(model, attacks, dataset_name)
                    att_loss = loss_fn(att_Y_hat, suc_labels)
                    att_suc_rate = (att_preds != suc_labels).sum() / len(att_preds)
                    att_acc = 1 - att_suc_rate
                    att_time = (time_end - time_start)/len(suc_imgs)

                    att_accu.add(att_acc, att_suc_rate, att_loss, att_norm, att_time)                    
            

            if eps == 0:
                att_suc_indices = att_preds == suc_labels
            else:
                att_suc_indices = att_preds != suc_labels
            examples_temp[eps] = {
                'suc_labels': suc_labels[att_suc_indices][0:num_examples].detach().cpu(),
                'suc_probs':suc_probs[att_suc_indices][0:num_examples].detach().cpu(),
                'att_preds':att_preds[att_suc_indices][0:num_examples].detach().cpu(),
                'att_probs':att_probs[att_suc_indices][0:num_examples].detach().cpu(),
                'imgs':attacks[att_suc_indices][0:num_examples].detach().cpu(),
                'deltas':deltas[att_suc_indices][0:num_examples].detach().cpu()
            }
            examples[att_name] = examples_temp

            num_batch = min(max_iter, len(test_data))
            metrics_temp[eps] = {
                '模型精度'  : att_accu[0]/num_batch,
                '攻击成功率': att_accu[1]/num_batch,
                '损失大小'  : att_accu[2]/num_batch,
                '扰动大小'  : att_accu[3]/num_batch,
                '攻击时间'  : att_accu[4]/num_batch
            }
            metrics[att_name] = metrics_temp
            # print(f'{att_name}-{eps}:\n{metrics_temp}')


            tb_writer.add_images(att_name, attacks, step)
            step += 1

            torch.cuda.empty_cache()
        
    plot_imgs(att_name, examples, epsilons)



### DeepFool

In [ ]:
if 'DeepFool' in att_modes:
    pass

### PGD

In [ ]:
if 'PGD' in att_modes:
    # 临时变量，临时保存数据与示例图片
    metrics_temp = {}
    examples_temp = {}

    # 新建攻击模型，并设置名称
    att_method = attack.PGD(model)
    att_name = att_method.__class__.__name__

    # tensorboard所需要的步数
    step = 0
    
    # 循环遍历攻击强度，得到每种强度下的相关数据
    with tqdm(epsilons, total = len(epsilons), leave = True) as t_eps:
        for eps in t_eps:
            # 定义相关参数累加器
            att_accu = utils.Accumulator(5)
            
            # 定义当前数据集批量迭代次数
            num_iters = 0
            with tqdm(test_data, total = max_iter, leave = False) as t_data:
                for imgs_in, labels in t_data:
                    # 当迭代次数超过设定值时，停止迭代，退出循环
                    num_iters += 1
                    if num_iters > max_iter: break
                    
                    # 提前预测一遍数据集，挑选出预测成功的例子用于攻击
                    raw_preds, raw_probs, Y_hat = utils.predict(model, imgs_in, dataset_name)

                    # 计算损失值
                    loss = loss_fn(Y_hat, labels)

                    # 获取预测成功的标签
                    suc_indices = raw_preds == labels
                    suc_imgs, suc_labels, suc_probs = imgs_in[suc_indices], labels[suc_indices], raw_probs[suc_indices]


                    #----------------------------------------#
                    # 开始攻击
                    #----------------------------------------#
                    
                    attacks = torch.zeros_like(suc_imgs)

                    # 计算时间
                    time_start = time.time()
                    # 攻击图片
                    attacks = att_method(suc_imgs, suc_labels, eps)
                    time_end = time.time()

                    # 得到扰动图片
                    deltas = attacks - suc_imgs
                    # 计算扰动的大小，L2范数
                    att_norm = torch.mean(
                    torch.linalg.norm(deltas.reshape(len(deltas), -1), dim=1) 
                        / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim=1)
                    )

                    # 计算攻击后的数据
                    att_preds, att_probs, att_Y_hat = utils.predict(model, attacks, dataset_name)
                    att_loss = loss_fn(att_Y_hat, suc_labels)
                    att_suc_rate = (att_preds != suc_labels).sum() / len(att_preds)
                    att_acc = 1 - att_suc_rate
                    att_time = (time_end - time_start)/len(suc_imgs)

                    # 累加相关数据，用于求平均值
                    att_accu.add(att_acc, att_suc_rate, att_loss, att_norm, att_time)                    
            
            if eps == 0:    # 保存eps为0的特例
                att_suc_indices = att_preds == suc_labels
            else:           # 保存eps不为0时，攻击成功的标号
                att_suc_indices = att_preds != suc_labels

            # 示例数据，保存为字典
            examples_temp[eps] = {
                'suc_labels': suc_labels[att_suc_indices][0:num_examples].detach().cpu(),
                'suc_probs':suc_probs[att_suc_indices][0:num_examples].detach().cpu(),
                'att_preds':att_preds[att_suc_indices][0:num_examples].detach().cpu(),
                'att_probs':att_probs[att_suc_indices][0:num_examples].detach().cpu(),
                'imgs':attacks[att_suc_indices][0:num_examples].detach().cpu(),
                'deltas':deltas[att_suc_indices][0:num_examples].detach().cpu()
            }
            # 按攻击方式保存示例
            examples[att_name] = examples_temp

            # tensorboard可视化
            tb_writer.add_images(att_name, attacks, step)
            step += 1

            # 保存相关数据
            num_batch = min(max_iter, len(test_data))
            metrics_temp[eps] = {
                '模型精度'  : att_accu[0]/num_batch,
                '攻击成功率': att_accu[1]/num_batch,
                '损失大小'  : att_accu[2]/num_batch,
                '扰动大小'  : att_accu[3]/num_batch,
                '攻击时间'  : att_accu[4]/num_batch
            }
            metrics[att_name] = metrics_temp
            # print(f'{att_name}-{eps}:\n{metrics_temp}')

            # 清理显存缓存，防止显存爆炸
            torch.cuda.empty_cache()

    # 显示图片
    plot_imgs(att_name, examples, epsilons)

### NIFGSM

In [ ]:
if 'NIFGSM' in att_modes:
    # 临时变量，临时保存数据与示例图片
    metrics_temp = {}
    examples_temp = {}

    # 新建攻击模型，并设置名称
    att_method = attack.NIFGSM(model)
    att_name = att_method.__class__.__name__

    # tensorboard所需要的步数
    step = 0
    
    # 循环遍历攻击强度，得到每种强度下的相关数据
    with tqdm(epsilons, total = len(epsilons), leave = True) as t_eps:
        for eps in t_eps:
            # 定义相关参数累加器
            att_accu = utils.Accumulator(5)
            
            # 定义当前数据集批量迭代次数
            num_iters = 0
            with tqdm(test_data, total = max_iter, leave = False) as t_data:
                for imgs_in, labels in t_data:
                    # 当迭代次数超过设定值时，停止迭代，退出循环
                    num_iters += 1
                    if num_iters > max_iter: break
                    
                    # 提前预测一遍数据集，挑选出预测成功的例子用于攻击
                    raw_preds, raw_probs, Y_hat = utils.predict(model, imgs_in, dataset_name)

                    # 计算损失值
                    loss = loss_fn(Y_hat, labels)

                    # 获取预测成功的标签
                    suc_indices = raw_preds == labels
                    suc_imgs, suc_labels, suc_probs = imgs_in[suc_indices], labels[suc_indices], raw_probs[suc_indices]


                    #----------------------------------------#
                    # 开始攻击
                    #----------------------------------------#
                    
                    attacks = torch.zeros_like(suc_imgs)

                    # 计算时间
                    time_start = time.time()
                    # 攻击图片
                    attacks = att_method(suc_imgs, suc_labels, eps)
                    time_end = time.time()

                    # 得到扰动图片
                    deltas = attacks - suc_imgs
                    # 计算扰动的大小，L2范数
                    att_norm = torch.mean(
                    torch.linalg.norm(deltas.reshape(len(deltas), -1), dim=1) 
                        / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim=1)
                    )

                    # 计算攻击后的数据
                    att_preds, att_probs, att_Y_hat = utils.predict(model, attacks, dataset_name)
                    att_loss = loss_fn(att_Y_hat, suc_labels)
                    att_suc_rate = (att_preds != suc_labels).sum() / len(att_preds)
                    att_acc = 1 - att_suc_rate
                    att_time = (time_end - time_start)/len(suc_imgs)

                    # 累加相关数据，用于求平均值
                    att_accu.add(att_acc, att_suc_rate, att_loss, att_norm, att_time)                    
            
            if eps == 0:    # 保存eps为0的特例
                att_suc_indices = att_preds == suc_labels
            else:           # 保存eps不为0时，攻击成功的标号
                att_suc_indices = att_preds != suc_labels

            # 示例数据，保存为字典
            examples_temp[eps] = {
                'suc_labels': suc_labels[att_suc_indices][0:num_examples].detach().cpu(),
                'suc_probs':suc_probs[att_suc_indices][0:num_examples].detach().cpu(),
                'att_preds':att_preds[att_suc_indices][0:num_examples].detach().cpu(),
                'att_probs':att_probs[att_suc_indices][0:num_examples].detach().cpu(),
                'imgs':attacks[att_suc_indices][0:num_examples].detach().cpu(),
                'deltas':deltas[att_suc_indices][0:num_examples].detach().cpu()
            }
            # 按攻击方式保存示例
            examples[att_name] = examples_temp

            # tensorboard可视化
            tb_writer.add_images(att_name, attacks, step)
            step += 1

            # 保存相关数据
            num_batch = min(max_iter, len(test_data))
            metrics_temp[eps] = {
                '模型精度'  : att_accu[0]/num_batch,
                '攻击成功率': att_accu[1]/num_batch,
                '损失大小'  : att_accu[2]/num_batch,
                '扰动大小'  : att_accu[3]/num_batch,
                '攻击时间'  : att_accu[4]/num_batch
            }
            metrics[att_name] = metrics_temp
            # print(f'{att_name}-{eps}:\n{metrics_temp}')

            # 清理显存缓存，防止显存爆炸
            torch.cuda.empty_cache()

    # 显示图片
    plot_imgs(att_name, examples, epsilons)

### 保存数据

In [ ]:
# torch.save(examples, att_examples_path + 'att_examples_' + str.join('_', att_modes))

metrics_df = pd.DataFrame.from_dict({(i, j): metrics[i][j] for i in metrics.keys() for j in metrics[i].keys()})
metrics_df.to_csv(att_metrics_path + 'att_metrics_' + str.join('_', att_modes) + '.csv', encoding='utf_8_sig')
metrics_df

## 泛化性能

### 定义参数

In [2]:
# 设置随机种子，使结果可复现
seed = 0
utils.set_random_seed(seed)

# 设置在GPU上运行
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 设置模型名称和路径
resnet_name = 'trans-resnet'
resnet_path = r'./results/CIFAR10/pre-resnet-bs128-lr0.01-seed0/models/pre-resnet-bs128-lr0.01-seed0'

mobilenet_name = 'trans-mobilenet'
mobilenet_path = r'./results/CIFAR10/pre-mobilenet-bs64-lr0.01-seed0/models/pre-mobilenet-bs64-lr0.01-seed0'

alexnet_name = 'trans-alexnet'
alexnet_path = r'./results/CIFAR10/alexnet-bs128-lr0.01-seed0/models/alexnet-bs128-lr0.01-seed0'

# 设置数据集相关参数
dataset_name = 'CIFAR10'# 数据集类型
data_channel = 3        # 图像通道数
batch_size = 100        # 图片批量大小
max_iter = 10           # 最大迭代的批量数量，减少程序运行时间

# 攻击强度范围和步长
epsilons = np.append(np.arange(0., 0.15, 0.015), [0.15, 0.2, 0.25, 0.3, 0.4, 0.5])
# epsilons = [0, 0.02]

# 设置结果参数
num_examples = 5        # 每种参数下的示例图片数量
metrics = {}            # 相关数据
examples = {}           # 示例图片

# 设置损失函数，用于计算损失值
loss_fn = nn.CrossEntropyLoss()

# 设置保存路径
att_results_path = f'./results_trans_attack/{dataset_name}'
att_metrics_path = f'{att_results_path}/metrics/'
att_examples_path = f'{att_results_path}/examples/'
att_tb_path = f'{att_results_path}/tb_out/'
paths = [att_metrics_path, att_tb_path, att_examples_path]
for path in paths:
    # 若路径已存在则不创建新文件夹
    if not os.path.exists(path):
        os.makedirs(path)
utils.check_path(att_results_path)

文件夹\metrics下存在文件['att_metrics_BIM.csv', 'att_metrics_FGSM.csv', 'att_metrics_NIFGSM.csv', 'att_metrics_PGD.csv', 'att_metrics__FGSM.csv']
文件夹./results_trans_attack/CIFAR10下已存在数据，继续运行将覆盖原有数据，是否继续运行？y/[n]
已删除"./results_trans_attack/CIFAR10/tb_out/"下的数据


### 创建待评估模型与数据集

In [3]:
# 读取路径下的模型
alexnet_model = utils.creat_model(alexnet_name, alexnet_path, in_ch=data_channel, pretrained=True)
mobilenet_model = utils.creat_model(mobilenet_name, mobilenet_path, in_ch=data_channel, pretrained=True)
resnet_model = utils.creat_model(resnet_name, resnet_path, in_ch=data_channel, pretrained=True)

models = [alexnet_model, mobilenet_model, resnet_model]

# 创建测试数据集
test_data = utils.data_iter(dataset_name, 'test', batch_size, 0)[1]

Files already downloaded and verified
Files already downloaded and verified


### FGSM

In [4]:
# 循环遍历三种网络
with tqdm(models, total = len(models), leave = True) as t_model:
    for model in t_model:
        # 新建攻击模型，并设置名称
        att_method = attack.FGSM(model)
        att_name = att_method.__class__.__name__

        metrics_tt = {}

        for test_model in models:
            # 临时变量，临时保存数据与示例图片
            metrics_temp = {}

            # 循环遍历攻击强度，得到每种强度下的相关数据
            with tqdm(epsilons, total = len(epsilons), leave = False) as t_eps:
                for eps in t_eps:
                    # 定义相关参数累加器
                    att_accu = utils.Accumulator(1)
                    
                    # 定义当前数据集批量迭代次数
                    num_iters = 0
                    for imgs_in, labels in test_data:
                        # 当迭代次数超过设定值时，停止迭代，退出循环
                        num_iters += 1
                        if num_iters > max_iter: break
                        
                        # 提前预测一遍数据集，挑选出预测成功的例子用于攻击
                        raw_preds, raw_probs, Y_hat = utils.predict(test_model, imgs_in, dataset_name)

                        # 计算损失值
                        loss = loss_fn(Y_hat, labels)

                        # 获取预测成功的标签
                        suc_indices = raw_preds == labels
                        suc_imgs, suc_labels, suc_probs = imgs_in[suc_indices], labels[suc_indices], raw_probs[suc_indices]


                        #----------------------------------------#
                        # 开始攻击
                        #----------------------------------------#
                        
                        attacks = torch.zeros_like(suc_imgs)

                        # 攻击图片
                        attacks = att_method(suc_imgs, suc_labels, eps)

                        # 得到扰动图片
                        deltas = attacks - suc_imgs
                        # 计算扰动的大小，L2范数
                        att_norm = torch.mean(
                        torch.linalg.norm(deltas.reshape(len(deltas), -1), dim=1) 
                            / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim=1)
                        )

                        # 计算攻击后的数据
                        att_preds, att_probs, att_Y_hat = utils.predict(test_model, attacks, dataset_name)
                        # att_loss = loss_fn(att_Y_hat, suc_labels)
                        att_suc_rate = (att_preds != suc_labels).sum() / len(att_preds)
                        # att_acc = 1 - att_suc_rate
                        # att_time = (time_end - time_start)/len(suc_imgs)

                        # 累加相关数据，用于求平均值
                        # att_accu.add(att_acc, att_suc_rate, att_loss, att_norm, att_time)
                        att_accu.add(att_suc_rate)
                    
                    if eps == 0:    # 保存eps为0的特例
                        att_suc_indices = att_preds == suc_labels
                    else:           # 保存eps不为0时，攻击成功的标号
                        att_suc_indices = att_preds != suc_labels

                    # 保存相关数据
                    num_batch = min(max_iter, len(test_data))
                    metrics_temp[eps] = att_accu[0] / num_batch

                    # 清理显存缓存，防止显存爆炸
                    torch.cuda.empty_cache()

                metrics_tt[test_model.name] = metrics_temp

        metrics[model.name] = metrics_tt


#----------------------------------------#
# 保存数据
#----------------------------------------#
metrics_df = pd.DataFrame.from_dict({(i, j): metrics[i][j] for i in metrics.keys() for j in metrics[i].keys()})
metrics_df.to_csv(att_metrics_path + 'att_metrics_' + att_name + '.csv', encoding='utf_8_sig')
metrics_df

  0%|          | 0/3 [00:00<?, ?it/s]

Using FGSM attack trans-alexnet on cuda.


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

...

Using FGSM attack trans-resnet on cuda.


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

trans-alexnet                              trans-mobilenet  \
      trans-alexnet trans-mobilenet trans-resnet   trans-alexnet   
0.000      0.064345        0.004409     0.002116        0.063060   
0.015      0.781101        0.157386     0.106803        0.080376   
0.030      0.864729        0.284331     0.240544        0.108494   
0.045      0.876020        0.395847     0.350350        0.147404   
0.060      0.882375        0.461940     0.445085        0.163087   
0.075      0.888219        0.533442     0.491746        0.224510   
0.090      0.894274        0.558502     0.546953        0.293870   
0.105      0.883501        0.617341     0.588732        0.350864   
0.120      0.906967        0.636746     0.633543        0.432697   
0.135      0.909395        0.677283     0.647943        0.512696   
0.150      0.901406        0.695011     0.650726        0.591771   
0.200      0.909454        0.749945     0.710420        0.700556   
0.250      0.910865        0.773419     0.752709        0.779752   
0.300      0.915375        0.809111     0.789411        0.829743   
0.400      0.896870        0.831672     0.813758        0.872738   
0.500      0.908030        0.849569     0.850431        0.903401   

                                    trans-resnet                               
      trans-mobilenet trans-resnet trans-alexnet trans-mobilenet trans-resnet  
0.000        0.004409     0.002116      0.062855        0.004409     0.002116  
0.015        0.335179     0.060279      0.059533        0.097137     0.225272  
0.030        0.422191     0.097655      0.120998        0.165825     0.248048  
0.045        0.538202     0.128821      0.172048        0.224625     0.280462  
0.060        0.611596     0.153732      0.234941        0.284691     0.309563  
0.075        0.684965     0.176775      0.349557        0.352405     0.331297  
0.090        0.722383     0.203673      0.423766        0.409588     0.350977  
0.105        0.746772     0.223373      0.522603        0.455647     0.387535  
0.120        0.768903     0.253531      0.594005        0.508317     0.413501  
0.135        0.779847     0.286749      0.653259        0.555476     0.437367  
0.150        0.793075     0.313810      0.714308        0.592085     0.481090  
0.200        0.821659     0.423004      0.808375        0.716712     0.574718  
0.250        0.837064     0.507135      0.847654        0.781148     0.666885  
0.300        0.847958     0.595648      0.857235        0.806846     0.726341  
0.400        0.874610     0.712986      0.903244        0.852222     0.826864  
0.500        0.887758     0.800326      0.901573        0.869704     0.862522

### BIM

In [5]:
# 循环遍历三种网络
with tqdm(models, total = len(models), leave = True) as t_model:
    for model in t_model:
        # 新建攻击模型，并设置名称
        att_method = attack.BIM(model)
        att_name = att_method.__class__.__name__

        metrics_tt = {}

        for test_model in models:
            # 临时变量，临时保存数据与示例图片
            metrics_temp = {}

            # 循环遍历攻击强度，得到每种强度下的相关数据
            with tqdm(epsilons, total = len(epsilons), leave = False) as t_eps:
                for eps in t_eps:
                    # 定义相关参数累加器
                    att_accu = utils.Accumulator(1)
                    
                    # 定义当前数据集批量迭代次数
                    num_iters = 0
                    for imgs_in, labels in test_data:
                        # 当迭代次数超过设定值时，停止迭代，退出循环
                        num_iters += 1
                        if num_iters > max_iter: break
                        
                        # 提前预测一遍数据集，挑选出预测成功的例子用于攻击
                        raw_preds, raw_probs, Y_hat = utils.predict(test_model, imgs_in, dataset_name)

                        # 计算损失值
                        loss = loss_fn(Y_hat, labels)

                        # 获取预测成功的标签
                        suc_indices = raw_preds == labels
                        suc_imgs, suc_labels, suc_probs = imgs_in[suc_indices], labels[suc_indices], raw_probs[suc_indices]


                        #----------------------------------------#
                        # 开始攻击
                        #----------------------------------------#
                        
                        attacks = torch.zeros_like(suc_imgs)

                        # 攻击图片
                        attacks = att_method(suc_imgs, suc_labels, eps)

                        # 得到扰动图片
                        deltas = attacks - suc_imgs
                        # 计算扰动的大小，L2范数
                        att_norm = torch.mean(
                        torch.linalg.norm(deltas.reshape(len(deltas), -1), dim=1) 
                            / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim=1)
                        )

                        # 计算攻击后的数据
                        att_preds, att_probs, att_Y_hat = utils.predict(test_model, attacks, dataset_name)
                        # att_loss = loss_fn(att_Y_hat, suc_labels)
                        att_suc_rate = (att_preds != suc_labels).sum() / len(att_preds)
                        # att_acc = 1 - att_suc_rate
                        # att_time = (time_end - time_start)/len(suc_imgs)

                        # 累加相关数据，用于求平均值
                        # att_accu.add(att_acc, att_suc_rate, att_loss, att_norm, att_time)
                        att_accu.add(att_suc_rate)
                    
                    if eps == 0:    # 保存eps为0的特例
                        att_suc_indices = att_preds == suc_labels
                    else:           # 保存eps不为0时，攻击成功的标号
                        att_suc_indices = att_preds != suc_labels

                    # 保存相关数据
                    num_batch = min(max_iter, len(test_data))
                    metrics_temp[eps] = att_accu[0] / num_batch

                    # 清理显存缓存，防止显存爆炸
                    torch.cuda.empty_cache()

                metrics_tt[test_model.name] = metrics_temp

        metrics[model.name] = metrics_tt


#----------------------------------------#
# 保存数据
#----------------------------------------#
metrics_df = pd.DataFrame.from_dict({(i, j): metrics[i][j] for i in metrics.keys() for j in metrics[i].keys()})
metrics_df.to_csv(att_metrics_path + 'att_metrics_' + att_name + '.csv', encoding='utf_8_sig')
metrics_df

  0%|          | 0/3 [00:00<?, ?it/s]

Using BIM attack trans-alexnet on cuda.


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

...

Using BIM attack trans-resnet on cuda.


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

trans-alexnet                              trans-mobilenet  \
      trans-alexnet trans-mobilenet trans-resnet   trans-alexnet   
0.000      0.057901        0.004409     0.002116        0.048330   
0.015      0.916333        0.147088     0.096482        0.064409   
0.030      0.983308        0.314894     0.229500        0.092998   
0.045      0.990609        0.429936     0.369596        0.089199   
0.060      0.996365        0.536767     0.484442        0.101375   
0.075      0.997633        0.625455     0.583642        0.104521   
0.090      0.997701        0.655985     0.648366        0.127364   
0.105      1.000000        0.718906     0.693012        0.151330   
0.120      1.000000        0.757238     0.771976        0.145014   
0.135      0.998795        0.769014     0.794093        0.170800   
0.150      1.000000        0.806093     0.805475        0.191758   
0.200      1.000000        0.850903     0.852644        0.284937   
0.250      1.000000        0.880334     0.879346        0.354508   
0.300      1.000000        0.914812     0.904432        0.469807   
0.400      1.000000        0.925747     0.933482        0.623965   
0.500      1.000000        0.924657     0.944901        0.723557   

                                    trans-resnet                               
      trans-mobilenet trans-resnet trans-alexnet trans-mobilenet trans-resnet  
0.000        0.004409     0.002116      0.056724        0.004409     0.002116  
0.015        0.611746     0.036083      0.054406        0.143468     0.932556  
0.030        0.808655     0.067357      0.094216        0.235539     0.990687  
0.045        0.909772     0.108757      0.098939        0.349350     0.996939  
0.060        0.957242     0.145308      0.138459        0.448086     0.998936  
0.075        0.984589     0.176558      0.172172        0.524787     0.997949  
0.090        0.988931     0.212627      0.190759        0.592956     0.998969  
0.105        0.987827     0.246008      0.254807        0.663829     1.000000  
0.120        0.995580     0.277080      0.312537        0.726586     1.000000  
0.135        0.995580     0.304347      0.372865        0.785691     0.998990  
0.150        0.994456     0.326786      0.402719        0.813450     0.998925  
0.200        0.997802     0.447636      0.577840        0.899188     1.000000  
0.250        0.997802     0.524970      0.717253        0.940124     1.000000  
0.300        0.998969     0.593421      0.781113        0.963438     1.000000  
0.400        1.000000     0.671939      0.879075        0.972350     1.000000  
0.500        1.000000     0.786935      0.926206        0.987739     1.000000

### PGD

In [6]:
# 循环遍历三种网络
with tqdm(models, total = len(models), leave = True) as t_model:
    for model in t_model:
        # 新建攻击模型，并设置名称
        att_method = attack.PGD(model)
        att_name = att_method.__class__.__name__

        metrics_tt = {}

        for test_model in models:
            # 临时变量，临时保存数据与示例图片
            metrics_temp = {}

            # 循环遍历攻击强度，得到每种强度下的相关数据
            with tqdm(epsilons, total = len(epsilons), leave = False) as t_eps:
                for eps in t_eps:
                    # 定义相关参数累加器
                    att_accu = utils.Accumulator(1)
                    
                    # 定义当前数据集批量迭代次数
                    num_iters = 0
                    for imgs_in, labels in test_data:
                        # 当迭代次数超过设定值时，停止迭代，退出循环
                        num_iters += 1
                        if num_iters > max_iter: break
                        
                        # 提前预测一遍数据集，挑选出预测成功的例子用于攻击
                        raw_preds, raw_probs, Y_hat = utils.predict(test_model, imgs_in, dataset_name)

                        # 计算损失值
                        loss = loss_fn(Y_hat, labels)

                        # 获取预测成功的标签
                        suc_indices = raw_preds == labels
                        suc_imgs, suc_labels, suc_probs = imgs_in[suc_indices], labels[suc_indices], raw_probs[suc_indices]


                        #----------------------------------------#
                        # 开始攻击
                        #----------------------------------------#
                        
                        attacks = torch.zeros_like(suc_imgs)

                        # 攻击图片
                        attacks = att_method(suc_imgs, suc_labels, eps)

                        # 得到扰动图片
                        deltas = attacks - suc_imgs
                        # 计算扰动的大小，L2范数
                        att_norm = torch.mean(
                        torch.linalg.norm(deltas.reshape(len(deltas), -1), dim=1) 
                            / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim=1)
                        )

                        # 计算攻击后的数据
                        att_preds, att_probs, att_Y_hat = utils.predict(test_model, attacks, dataset_name)
                        # att_loss = loss_fn(att_Y_hat, suc_labels)
                        att_suc_rate = (att_preds != suc_labels).sum() / len(att_preds)
                        # att_acc = 1 - att_suc_rate
                        # att_time = (time_end - time_start)/len(suc_imgs)

                        # 累加相关数据，用于求平均值
                        # att_accu.add(att_acc, att_suc_rate, att_loss, att_norm, att_time)
                        att_accu.add(att_suc_rate)
                    
                    if eps == 0:    # 保存eps为0的特例
                        att_suc_indices = att_preds == suc_labels
                    else:           # 保存eps不为0时，攻击成功的标号
                        att_suc_indices = att_preds != suc_labels

                    # 保存相关数据
                    num_batch = min(max_iter, len(test_data))
                    metrics_temp[eps] = att_accu[0] / num_batch

                    # 清理显存缓存，防止显存爆炸
                    torch.cuda.empty_cache()

                metrics_tt[test_model.name] = metrics_temp

        metrics[model.name] = metrics_tt


#----------------------------------------#
# 保存数据
#----------------------------------------#
metrics_df = pd.DataFrame.from_dict({(i, j): metrics[i][j] for i in metrics.keys() for j in metrics[i].keys()})
metrics_df.to_csv(att_metrics_path + 'att_metrics_' + att_name + '.csv', encoding='utf_8_sig')
metrics_df

  0%|          | 0/3 [00:00<?, ?it/s]

Using PGD attack trans-alexnet on cuda.


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

...

Using PGD attack trans-resnet on cuda.


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

trans-alexnet                              trans-mobilenet  \
      trans-alexnet trans-mobilenet trans-resnet   trans-alexnet   
0.000      0.049857        0.004409     0.002116        0.046060   
0.015      0.917393        0.140112     0.102818        0.067519   
0.030      0.981114        0.318396     0.231181        0.061441   
0.045      0.993957        0.420931     0.366355        0.075425   
0.060      0.997647        0.551095     0.496595        0.103357   
0.075      1.000000        0.625406     0.596684        0.098165   
0.090      0.998810        0.654056     0.650093        0.130144   
0.105      1.000000        0.737169     0.703319        0.131389   
0.120      1.000000        0.758048     0.752071        0.157468   
0.135      0.998684        0.765061     0.784922        0.157833   
0.150      1.000000        0.815522     0.805490        0.190914   
0.200      1.000000        0.846887     0.854510        0.256473   
0.250      1.000000        0.884779     0.876584        0.345287   
0.300      1.000000        0.911101     0.920733        0.441348   
0.400      1.000000        0.931541     0.928507        0.610000   
0.500      1.000000        0.918245     0.931384        0.696187   

                                    trans-resnet                               
      trans-mobilenet trans-resnet trans-alexnet trans-mobilenet trans-resnet  
0.000        0.004409     0.002116      0.061742        0.004409     0.002116  
0.015        0.614893     0.036061      0.089874        0.144530     0.931546  
0.030        0.803765     0.070417      0.089597        0.234318     0.985553  
0.045        0.908759     0.111981      0.105253        0.339374     0.997969  
0.060        0.961626     0.140180      0.139191        0.458207     0.996895  
0.075        0.986832     0.173460      0.175824        0.534448     0.996873  
0.090        0.991105     0.215621      0.201807        0.602162     0.998969  
0.105        0.990115     0.247236      0.274141        0.682569     1.000000  
0.120        0.994504     0.274793      0.306707        0.717972     1.000000  
0.135        0.995580     0.307540      0.381821        0.787264     0.998990  
0.150        0.994469     0.318774      0.428766        0.823746     1.000000  
0.200        0.996703     0.447667      0.588422        0.909035     1.000000  
0.250        0.997802     0.513956      0.706755        0.927111     1.000000  
0.300        1.000000     0.577894      0.790654        0.961175     1.000000  
0.400        0.997778     0.688132      0.864935        0.972180     1.000000  
0.500        1.000000     0.783055      0.899428        0.982088     1.000000

### NIFGSM

In [7]:
# 循环遍历三种网络
with tqdm(models, total = len(models), leave = True) as t_model:
    for model in t_model:
        # 新建攻击模型，并设置名称
        att_method = attack.NIFGSM(model)
        att_name = att_method.__class__.__name__

        metrics_tt = {}

        for test_model in models:
            # 临时变量，临时保存数据与示例图片
            metrics_temp = {}

            # 循环遍历攻击强度，得到每种强度下的相关数据
            with tqdm(epsilons, total = len(epsilons), leave = False) as t_eps:
                for eps in t_eps:
                    # 定义相关参数累加器
                    att_accu = utils.Accumulator(1)
                    
                    # 定义当前数据集批量迭代次数
                    num_iters = 0
                    for imgs_in, labels in test_data:
                        # 当迭代次数超过设定值时，停止迭代，退出循环
                        num_iters += 1
                        if num_iters > max_iter: break
                        
                        # 提前预测一遍数据集，挑选出预测成功的例子用于攻击
                        raw_preds, raw_probs, Y_hat = utils.predict(test_model, imgs_in, dataset_name)

                        # 计算损失值
                        loss = loss_fn(Y_hat, labels)

                        # 获取预测成功的标签
                        suc_indices = raw_preds == labels
                        suc_imgs, suc_labels, suc_probs = imgs_in[suc_indices], labels[suc_indices], raw_probs[suc_indices]


                        #----------------------------------------#
                        # 开始攻击
                        #----------------------------------------#
                        
                        attacks = torch.zeros_like(suc_imgs)

                        # 攻击图片
                        attacks = att_method(suc_imgs, suc_labels, eps)

                        # 得到扰动图片
                        deltas = attacks - suc_imgs
                        # 计算扰动的大小，L2范数
                        att_norm = torch.mean(
                        torch.linalg.norm(deltas.reshape(len(deltas), -1), dim=1) 
                            / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim=1)
                        )

                        # 计算攻击后的数据
                        att_preds, att_probs, att_Y_hat = utils.predict(test_model, attacks, dataset_name)
                        # att_loss = loss_fn(att_Y_hat, suc_labels)
                        att_suc_rate = (att_preds != suc_labels).sum() / len(att_preds)
                        # att_acc = 1 - att_suc_rate
                        # att_time = (time_end - time_start)/len(suc_imgs)

                        # 累加相关数据，用于求平均值
                        # att_accu.add(att_acc, att_suc_rate, att_loss, att_norm, att_time)
                        att_accu.add(att_suc_rate)
                    
                    if eps == 0:    # 保存eps为0的特例
                        att_suc_indices = att_preds == suc_labels
                    else:           # 保存eps不为0时，攻击成功的标号
                        att_suc_indices = att_preds != suc_labels

                    # 保存相关数据
                    num_batch = min(max_iter, len(test_data))
                    metrics_temp[eps] = att_accu[0] / num_batch

                    # 清理显存缓存，防止显存爆炸
                    torch.cuda.empty_cache()

                metrics_tt[test_model.name] = metrics_temp

        metrics[model.name] = metrics_tt


#----------------------------------------#
# 保存数据
#----------------------------------------#
metrics_df = pd.DataFrame.from_dict({(i, j): metrics[i][j] for i in metrics.keys() for j in metrics[i].keys()})
metrics_df.to_csv(att_metrics_path + 'att_metrics_' + att_name + '.csv', encoding='utf_8_sig')
metrics_df

  0%|          | 0/3 [00:00<?, ?it/s]

Using NIFGSM attack trans-alexnet on cuda.


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

...

Using NIFGSM attack trans-resnet on cuda.


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

trans-alexnet                              trans-mobilenet  \
      trans-alexnet trans-mobilenet trans-resnet   trans-alexnet   
0.000      0.050122        0.004409     0.002116        0.063017   
0.015      0.930283        0.198801     0.143161        0.081040   
0.030      0.974642        0.392404     0.344726        0.090468   
0.045      0.990225        0.568121     0.553465        0.127541   
0.060      0.994073        0.697440     0.696197        0.156439   
0.075      0.994143        0.781590     0.779170        0.190070   
0.090      0.998765        0.802375     0.834620        0.223401   
0.105      1.000000        0.834137     0.879148        0.265214   
0.120      0.998810        0.861086     0.881243        0.331767   
0.135      1.000000        0.890412     0.906213        0.359444   
0.150      0.998750        0.896445     0.919888        0.410288   
0.200      1.000000        0.932286     0.931628        0.570090   
0.250      1.000000        0.931332     0.948159        0.656079   
0.300      0.998795        0.936660     0.948158        0.767323   
0.400      1.000000        0.917639     0.959511        0.846572   
0.500      1.000000        0.937142     0.963733        0.887675   

                                    trans-resnet                               
      trans-mobilenet trans-resnet trans-alexnet trans-mobilenet trans-resnet  
0.000        0.004409     0.002116      0.048757        0.004409     0.002116  
0.015        0.650387     0.071460      0.072762        0.181437     0.926140  
0.030        0.846505     0.145177      0.119592        0.355583     0.990730  
0.045        0.938662     0.229510      0.218999        0.490131     0.998969  
0.060        0.980384     0.267776      0.237787        0.621679     0.997959  
0.075        0.987936     0.330237      0.358690        0.717903     1.000000  
0.090        0.992409     0.384660      0.456298        0.784208     1.000000  
0.105        0.997802     0.434446      0.535914        0.832185     1.000000  
0.120        0.994573     0.472831      0.625679        0.880349     1.000000  
0.135        0.996703     0.512136      0.669021        0.900892     1.000000  
0.150        0.997802     0.545362      0.714295        0.913430     1.000000  
0.200        0.998901     0.627823      0.823680        0.935776     1.000000  
0.250        0.998901     0.734851      0.860233        0.956942     1.000000  
0.300        1.000000     0.795175      0.905587        0.966739     1.000000  
0.400        1.000000     0.890221      0.942610        0.988947     1.000000  
0.500        0.998901     0.938715      0.937743        0.989935     1.000000